In [59]:
import pandas as pd

In [96]:
df = pd.read_csv('건설업전처리완료(target수정).csv')

In [97]:
df.columns

Index(['ID', 'SQ1', 'SQ2', 'Q1_1A', 'Q1_1B', 'Q1_1C', 'Q1_2A', 'Q1_2B',
       'Q1_2C', 'Q1_3', 'Q1_4', 'Q2', 'Q3_1_1', 'Q4_1', 'Q4_2', 'Q4_3', 'Q4_4',
       'Q5', 'Q6', 'Q9', 'Q11', 'Q13_1', 'Q13_2', 'Q13_3', 'Q13_4', 'Q13_5',
       'Q13_6', 'Q13_7', 'Q13_8', 'Q13_9', 'Q13_10', 'Q13_11', 'Q13_12',
       'Q13_13', 'Q13_14', 'Q13_15', 'Q13_16', 'Q13_17', 'Q13_18', 'Q13_19',
       'Q13_20', 'Q16_1', 'Q16_4', 'Q16_13', 'Q16_14', 'Q16_15', 'Q22',
       'Q28_1', 'target', 'duration', '55_ratio', '55_plus', '55_mul',
       'foreign_ratio', 'women_ratio', 'syear_type', 'year'],
      dtype='object')

In [98]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

import optuna
from optuna import Trial
from optuna.samplers import TPESampler

In [102]:
y = df.target
X = df.drop(columns='target')

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

scores = []
oof_pred = np.zeros(X_test.shape[0])  # OOF 저장
kfold = KFold(n_splits=13, shuffle=True, random_state=0)

for train_index, valid_index in kfold.split(X_train, y_train):
    train_x, valid_x = X_train.iloc[train_index], X_train.iloc[valid_index]
    train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]

    # LightGBM 모델링
    model = LGBMClassifier(random_state=0)
    model.fit(train_x, train_y,
              eval_set=[(valid_x, valid_y)],
              early_stopping_rounds=100,
              verbose=True
              )

    y_pred = model.predict_proba(valid_x)[:, 1]
    auc = roc_auc_score(valid_y, y_pred)
    scores.append(auc)

    oof_pred += model.predict_proba(X_test)[:, 1] / kfold.get_n_splits()

In [104]:
scores = np.array(scores) 
print("CV scores: ", scores)
print("CV mean = %.2f" % scores.mean(), "with std = %.2f" % scores.std())

CV scores:  [0.78322581 0.80185185 0.74816176 0.82412399 0.83264746 0.77055703
 0.81432361 0.79225806 0.76223776 0.79281046 0.72903226 0.76888021
 0.80730223]
CV mean = 0.79 with std = 0.03


In [105]:
from sklearn.model_selection import KFold
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

scores = []
oof_pred = np.zeros(X_test.shape[0])  # OOF 저장
kfold = KFold(n_splits=13, shuffle=True, random_state=0)

for train_index, valid_index in kfold.split(X_train, y_train):
    train_x, valid_x = X_train.iloc[train_index], X_train.iloc[valid_index]
    train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]

    # RandomForest 모델링
    model = RandomForestClassifier(random_state=0)
    model.fit(train_x, train_y)

    y_pred = model.predict_proba(valid_x)[:, 1]
    auc = roc_auc_score(valid_y, y_pred)
    scores.append(auc)

    oof_pred += model.predict_proba(X_test)[:, 1] / kfold.get_n_splits()


In [106]:
scores = np.array(scores) 
print("CV scores: ", scores)
print("CV mean = %.2f" % scores.mean(), "with std = %.2f" % scores.std())

CV scores:  [0.77129032 0.81419753 0.69852941 0.8143531  0.83367627 0.75928382
 0.82228117 0.76870968 0.7527972  0.80915033 0.76290323 0.79003906
 0.81000676]
CV mean = 0.79 with std = 0.04


In [107]:
from sklearn.model_selection import KFold
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

scores = []
oof_pred = np.zeros(X_test.shape[0])  # OOF 저장
kfold = KFold(n_splits=13, shuffle=True, random_state=0)

for train_index, valid_index in kfold.split(X_train, y_train):
    train_x, valid_x = X_train.iloc[train_index], X_train.iloc[valid_index]
    train_y, valid_y = y_train.iloc[train_index], y_train.iloc[valid_index]

    # XGBoost 모델링
    model = XGBClassifier(random_state=0)
    model.fit(train_x, train_y)

    y_pred = model.predict_proba(valid_x)[:, 1]
    auc = roc_auc_score(valid_y, y_pred)
    scores.append(auc)

    oof_pred += model.predict_proba(X_test)[:, 1] / kfold.get_n_splits()


In [108]:
scores = np.array(scores) 
print("CV scores: ", scores)
print("CV mean = %.2f" % scores.mean(), "with std = %.2f" % scores.std())

CV scores:  [0.73806452 0.73888889 0.75091912 0.78975741 0.81961591 0.74204244
 0.77785146 0.73032258 0.73776224 0.76339869 0.69419355 0.76041667
 0.74171738]
CV mean = 0.75 with std = 0.03


### Modeling(튜닝)

In [50]:
import optuna
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

class CatBoostLGBMRFXGB:
    def __init__(self, X_train, y_train, X_test, y_test):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.cat_study = None
        self.lgb_study = None
        self.rf_study = None
        self.xgb_study = None
        self.cat_best_params = None
        self.lgb_best_params = None
        self.rf_best_params = None
        self.xgb_best_params = None

    def cat_objective(self, trial):
        params_cat = {
            'iterations': trial.suggest_int('iterations', 100, 500),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
            'random_strength': trial.suggest_int('random_strength', 0, 100),
            'bagging_temperature': trial.suggest_int('bagging_temperature', 0, 100),
            'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
            'od_wait': trial.suggest_int('od_wait', 10, 50)
        }

        model = CatBoostClassifier(**params_cat, random_state=42)
        scores = cross_val_score(model, self.X_train, self.y_train, scoring='roc_auc', n_jobs=-1, cv=5)
        score = sum(scores) / len(scores)

        return score

    def lgb_objective(self, trial):
        params_lgb = {
            'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
            'max_depth': trial.suggest_int('max_depth', 6, 20),
            'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
            'n_estimators': trial.suggest_int('n_estimators', 1500, 3000, step=1),
            'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
            'subsample': trial.suggest_uniform('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
            'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 1.0),
        }

        model = LGBMClassifier(random_state=42, **params_lgb)
        scores = cross_val_score(model, self.X_train, self.y_train, scoring='roc_auc', n_jobs=-1, cv=5)
        score = sum(scores) / len(scores)

        return score
    
    def rf_objective(self, trial):
        params_rf = {
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 500),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'n_estimators':  trial.suggest_int('n_estimators', 100, 1000)
        }

        model = RandomForestClassifier(random_state=42, **params_rf)
        scores = cross_val_score(model, self.X_train, self.y_train, scoring='roc_auc', n_jobs=-1, cv=5)
        score = sum(scores) / len(scores)

        return score

    def xgb_objective(self, trial):
        params = {
            "max_depth": trial.suggest_int("max_depth", 6, 16),
            "learning_rate": trial.suggest_float('learning_rate', 0.05, 0.99),
            'n_estimators': trial.suggest_int("n_estimators", 1000, 10000, step=100),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
            "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 1, log=True),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 1, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0, step=0.05),
            'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
            "gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True),
        }

        model = XGBClassifier(random_state=42, **params)
        scores = cross_val_score(model, self.X_train, self.y_train, scoring='roc_auc', n_jobs=-1, cv=5)
        score = sum(scores) / len(scores)

        return score

    def optimize_catboost(self, n_trials=200):
        self.cat_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=42), direction='maximize')
        self.cat_study.optimize(self.cat_objective, n_trials=n_trials)
        self.cat_best_params = self.cat_study.best_params

    def optimize_lgbm(self, n_trials=200):
        self.lgb_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=42), direction='maximize')
        self.lgb_study.optimize(self.lgb_objective, n_trials=n_trials)
        self.lgb_best_params = self.lgb_study.best_params
    
    def optimize_rf(self, n_trials=200):
        self.rf_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=42), direction='maximize')
        self.rf_study.optimize(self.rf_objective, n_trials=n_trials)
        self.rf_best_params = self.rf_study.best_params
    
    def optimize_xgb(self, n_trials=200):
        self.xgb_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=42), direction='maximize')
        self.xgb_study.optimize(self.xgb_objective, n_trials=n_trials)
        self.xgb_best_params = self.xgb_study.best_params

    def train_catboost(self):
        CAT = CatBoostClassifier(**self.cat_best_params, random_state=42, verbose=False)
        CAT.fit(self.X_train, self.y_train)

    def train_lgbm(self):
        LGBM = LGBMClassifier(**self.lgb_best_params, random_state=42)
        LGBM.fit(self.X_train, self.y_train)
    
    def train_rf(self):
        RF = RandomForestClassifier(**self.rf_best_params, random_state=42)
        RF.fit(self.X_train, self.y_train)

    def train_xgb(self):
        XGB = XGBClassifier(**self.xgb_best_params, random_state=42)
        XGB.fit(self.X_train, self.y_train)

    def predict_catboost(self):
        CAT = CatBoostClassifier(**self.cat_best_params, random_state=42, verbose=False)
        CAT.fit(self.X_train, self.y_train)
        cat_pred = CAT.predict_proba(self.X_test)[:, 1]
        score = roc_auc_score(self.y_test, cat_pred)
        return score

    def predict_lgbm(self):
        LGBM = LGBMClassifier(**self.lgb_best_params, random_state=42)
        LGBM.fit(self.X_train, self.y_train)
        lgbm_pred = LGBM.predict_proba(self.X_test)[:, 1]
        score = roc_auc_score(self.y_test, lgbm_pred)
        return score
    
    def predict_rf(self):
        RF = RandomForestClassifier(**self.rf_best_params, random_state=42)
        RF.fit(self.X_train, self.y_train)
        rf_pred = RF.predict_proba(self.X_test)[:, 1]
        score = roc_auc_score(self.y_test, rf_pred)
        return score
    
    def predict_xgb(self):
        XGB = XGBClassifier(**self.xgb_best_params, random_state=42)
        XGB.fit(self.X_train, self.y_train)
        xgb_pred = XGB.predict_proba(self.X_test)[:, 1]
        score = roc_auc_score(self.y_test, xgb_pred)
        return score


In [ ]:
clf = CatBoostLGBMRFXGB(X_train, y_train, X_test, y_test)
clf.optimize_catboost(n_trials=200)
clf.optimize_lgbm(n_trials=200)
clf.optimize_rf(n_trials=200)
clf.optimize_xgb(n_trials=200)
clf.train_catboost()
clf.train_lgbm()
clf.train_rf()
clf.train_xgb()
catboost_score = clf.predict_catboost()
lgbm_score = clf.predict_lgbm()
rf_score = clf.predict_rf()
xgb_score = clf.predict_xgb()
print("CatBoost Score:", catboost_score)
print("LightGBM Score:", lgbm_score)
print("Random Forest Score:", rf_score)
print("XGBoost Score:", xgb_score)